<a href="https://www.kaggle.com/code/mubashirjawad01/raw-fatal-pcg-mother-and-child-dataset?scriptVersionId=298260624" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [8]:
import os
import glob
import scipy.io
import matplotlib.pyplot as plt
import numpy as np

# **Data Load**

In [39]:
folder_path = '/kaggle/input/datasets/mubashirjawad01/raw-fetal-pcg-contaminated-with-mothers-pcg' 

#searching for sub folder of.mat  when recursive = true  
search_pattern = os.path.join(folder_path, '**', '*.mat')
mat_files = glob.glob(search_pattern, recursive=True)

#data save dictonary 
all_data = {}

print(f"Total .mat files found: {len(mat_files)}\n")

#loading the file
if len(mat_files) > 0:
    for file_path in mat_files:
        file_name = os.path.basename(file_path)
        try:
            all_data[file_name] = scipy.io.loadmat(file_path)
            print(f" Loaded: {file_name}")
        except Exception as e:
            print(f"❌ Error loading {file_name}: {e}")
            
    print("\n All files are loaded succesfully ")
else:
    print("❌ didn't Find Any Folder")

Total .mat files found: 21

 Loaded: B2_P4_S1.mat
 Loaded: B1_P3_S2.mat
 Loaded: B2_P4_S3.mat
 Loaded: B1_P3_S1.mat
 Loaded: B2_P5_S1.mat
 Loaded: B2_P1_S3.mat
 Loaded: B2_P3_S1.mat
 Loaded: B2_P3_S2.mat
 Loaded: B2_P2_S2.mat
 Loaded: B2_P3_S3.mat
 Loaded: B2_P4_S2.mat
 Loaded: B2_P1_S2.mat
 Loaded: B1_P2_S1.mat
 Loaded: B2_P5_S2.mat
 Loaded: B2_P2_S1.mat
 Loaded: B2_P2_S3.mat
 Loaded: B1_P2_S2.mat
 Loaded: B2_P1_S1.mat
 Loaded: B2_P5_S3.mat
 Loaded: B1_P1_S2.mat
 Loaded: B1_P1_S1.mat

 All files are loaded succesfully 


In [32]:
print(df.columns)

Index(['B2_P4_S1.mat', 'B1_P3_S2.mat', 'B2_P4_S3.mat', 'B1_P3_S1.mat',
       'B2_P5_S1.mat', 'B2_P1_S3.mat', 'B2_P3_S1.mat', 'B2_P3_S2.mat',
       'B2_P2_S2.mat', 'B2_P3_S3.mat', 'B2_P4_S2.mat', 'B2_P1_S2.mat',
       'B1_P2_S1.mat', 'B2_P5_S2.mat', 'B2_P2_S1.mat', 'B2_P2_S3.mat',
       'B1_P2_S2.mat', 'B2_P1_S1.mat', 'B2_P5_S3.mat', 'B1_P1_S2.mat',
       'B1_P1_S1.mat'],
      dtype='object')


# **RGB and HSV Histogram Plot**

In [40]:
import matplotlib.pyplot as plt
import cv2

# ১. আগে চেক করে নেওয়া যে image_stats এ ডেটা আছে কি না
if len(all_data) == 0:
    print("❌ Error: 'image_stats' লিস্টটি খালি! আপনার আগের স্পেকট্রোগ্রাম জেনারেশন সেলটি আবার রান করুন।")
else:
    # ২. ডেটা থাকলে প্রথম ফাইলটির হিস্টোগ্রাম দেখানো
    sample = image_stats[0]
    
    # নিশ্চিত হওয়া যে ডিকশনারিতে ইমেজ ডেটা আছে
    if 'img_rgb' in sample and 'img_hsv' in sample:
        fig, axs = plt.subplots(1, 2, figsize=(15, 5))

        # RGB Histogram
        colors = ('r', 'g', 'b')
        for i, col in enumerate(colors):
            hist = cv2.calcHist([sample['img_rgb']], [i], None, [256], [0, 256])
            axs[0].plot(hist, color=col)
        axs[0].set_title(f"RGB Histogram - {sample['file']}")
        axs[0].set_xlabel("Pixel Intensity")
        axs[0].set_ylabel("Frequency")

        # HSV Histogram (Saturation channel)
        # index 1 হচ্ছে Saturation channel
        hist_s = cv2.calcHist([sample['img_hsv']], [1], None, [256], [0, 256])
        axs[1].plot(hist_s, color='orange')
        axs[1].set_title(f"Saturation (HSV) Histogram - {sample['file']}")
        axs[1].set_xlabel("Saturation Value")
        axs[1].set_ylabel("Frequency")

        plt.tight_layout()
        plt.show()
    else:
        print(f"❌ Error: {sample['file']} এর ভেতর ইমেজ ডেটা (RGB/HSV) পাওয়া যায়নি।")

IndexError: list index out of range